In [ ]:
import pandas as pd
import numpy as np
import UsefulFunctions as UF
import seaborn as sns
import matplotlib.pyplot as plt
import os

root = "C:\\Users\\b01297ar\\Documents\\ProstateMRL-local\\ProstateMRL-Radiomics\\Data\\Ttest\\"

files = os.listdir(root + "Tries\\")
df_all = pd.DataFrame()

for f in files:
    df_t = pd.read_csv(root + "Tries\\" + f)
    # drop first column
    #df_t = df_t.drop(df.columns[0], axis=1)
    df_all = df_all.append(df_t, ignore_index = True)


df_all = df_all.drop(df_all.columns[0], axis=1)


In [ ]:
# get average number of tries for each startTval
df_avg_mn = df_all.groupby("StartTval").mean()
df_avg_sd = df_all.groupby("StartTval").std()
# only keep the number of tries column
#df_avg.set_index("StartTval", inplace = True)
df_avg_mn = df_avg_mn["Tries"]
df_avg_sd = df_avg_sd["Tries"]
df_avg = pd.DataFrame()
df_avg["Mean"] = df_avg_mn
df_avg["Sd"] = df_avg_sd
df_avg.index.name = "StartTval"
#print(df_avg)

# plot
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(12, 8))
sns.lineplot(x=df_avg.index, y=df_avg.Mean, marker="o", color = "blue", legend=False)
sns.lineplot(x=df_avg.index, y=df_avg.Sd, marker="o", color = "red", legend=False)
plt.xlabel("Start Tval", fontsize = 20)
plt.xticks(np.arange(1, 3.01, step=0.25))
plt.ylabel("Attempts", fontsize = 20)
plt.legend(["Mean", "Sd"], fontsize = "20")
plt.title("Number of attempts to cluster", fontsize = 30)
plt.show()



In [ ]:


# t-vals to colours for plotting
tvals = np.arange(1, 3.01, step=0.25)
colours = sns.color_palette("colorblind", len(tvals))

df_plot = df_all.copy()

# violin plot
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(12, 8))
sns.violinplot(x="StartTval", y="Tries", hue="StartTval", palette=colours, data=df_plot)
plt.xlabel("Start Tval", fontsize = 20)
#plt.xticks(np.arange(1, 3.01, step=0.25))
plt.ylabel("Attempts", fontsize = 20)
#plt.legend(fontsize = "16", title = "Start Tval", title_fontsize = "20", loc = "upper right")
plt.title("Number of attempts to cluster", fontsize = 30)
plt.show()



In [ ]:
df_plot2 = df_all.copy()

# groupy by startTval and patID and get mean
df_plot2 = df_plot2.groupby(["StartTval", "PatID"]).mean()
df_plot2 = df_plot2.reset_index()
#df_plot2 = df_plot2.drop(df_plot2.columns[0], axis=1)
#df_plot2.columns = ["StartTval", "PatID", "Tries"]
print(df_plot2["StartTval"].unique())
df_plot2["StartTval"] = df_plot2["StartTval"].astype(float)
# violin plot
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(12, 8))
ax = sns.boxplot(x="StartTval", y="Tries", hue="StartTval", palette=colours, data=df_plot2)

plt.legend().remove()
plt.xlabel("Start Tval", fontsize = 20)
#plt.xlim(0.5, 3.5)
# ax.set_xticks(np.arange(1, 3.01, step=0.25))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, fontsize=16)
#plt.xticks(fontsize = 16, rotation = 45)
plt.ylabel("Attempts", fontsize = 20)
#plt.legend(fontsize = "16", title = "Start Tval", title_fontsize = "20", loc = "upper right")
plt.title("Number of attempts to cluster", fontsize = 30)
plt.show()

In [ ]:
# load in data
labels_dir = "C:\\Users\\b01297ar\\Documents\\ProstateMRL-local\\ProstateMRL-Radiomics\\Data\\Ttest\\ClusterLabels\\"

files = os.listdir(labels_dir)
df_all_vals = pd.DataFrame()
for file in files:
    t_val = float(file.split("_")[2][:-4])

    df_temp = pd.read_csv(labels_dir + file)
    df_temp["Tval"] = t_val

    df_all_vals = df_all_vals.append(df_temp, ignore_index = True)

df_all_vals = df_all_vals.drop(["Unnamed: 0", "Scan", "Mask", "Days", "FeatureChange"], axis=1)

In [ ]:
# show random sample of data
print(df_all_vals.sample(5))

patIDs = df_all_vals["PatID"].unique()
tvals = df_all_vals["Tval"].unique()


In [ ]:
df_fts = df_all_vals.loc[df_all_vals.Fraction == 1]
df_fts = df_fts.drop(["Fraction", "FeatureValue"], axis=1)
df_cl = df_fts.copy()
# get number of clusters per patient and set as new column
df_cl["NumCluster"] = df_cl.groupby(["PatID", "Tval"])["Cluster"].transform("nunique")
df_cl = df_cl.drop(["Cluster"], axis=1)
# get mean number of numfts column
df_cl = df_cl.groupby(["PatID", "Tval", "NumCluster"]).mean()
df_cl.rename(columns={"NumFts": "MeanNumFts"}, inplace=True)
df_cl = df_cl.reset_index()

In [ ]:
# violin plot of number of clusters per tval
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(12, 8))
sns.boxplot(x="Tval", y="NumCluster", hue="Tval", palette=colours, data=df_cl)
plt.xlabel("Start Tval", fontsize = 20)
#plt.xticks(np.arange(1, 3.01, step=0.25))
plt.ylabel("Number of clusters", fontsize = 20)
#plt.legend(fontsize = "16", title = "Start Tval", title_fontsize = "20", loc = "upper right")
plt.title("Number of clusters per patient", fontsize = 30)
plt.show()

In [ ]:
# violin plot of number of features per cluster per tval
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(12, 8))
sns.boxplot(x="Tval", y="MeanNumFts", hue="Tval", palette=colours, data=df_cl)
plt.xlabel("Start Tval", fontsize = 20)
#plt.xticks(np.arange(1, 3.01, step=0.25))
plt.ylabel("Number of features", fontsize = 20)
#plt.legend(fontsize = "16", title = "Start Tval", title_fontsize = "20", loc = "upper right")
plt.title("Number of features per cluster", fontsize = 30)
plt.show()

In [ ]:
df_cl2 = df_cl.copy()
df_cl2["FtsPerCluster"] = df_cl2["MeanNumFts"] / df_cl2["NumCluster"]
# violin plot of number of features per cluster per tval
sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.figure(figsize=(12, 8))
sns.boxplot(x="Tval", y="FtsPerCluster", hue="Tval", palette=colours, data=df_cl2)
plt.xlabel("Start Tval", fontsize = 20)
#plt.xticks(np.arange(1, 3.01, step=0.25))
plt.ylabel("Number of features", fontsize = 20)
#plt.legend(fontsize = "16", title = "Start Tval", title_fontsize = "20", loc = "upper right")

plt.title("Number of features per cluster", fontsize = 30)
plt.show()